# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.9/320.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [4]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data ={'table': ["Users", "Orders", "Products"], 
    'definition': [
        "Contains user details (id, name, email)", 
        "Stores order information (order_id, user_id, amount)",
        "Product catalog (product_id, name, price)"]
}
df = pd.DataFrame(data)
print(df)

      table                                         definition
0     Users            Contains user details (id, name, email)
1    Orders  Stores order information (order_id, user_id, a...
2  Products          Product catalog (product_id, name, price)


In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [6]:
print(text_tables)

Users: Contains user details (id, name, email)
Orders: Stores order information (order_id, user_id, amount)
Products: Product catalog (product_id, name, price)


In [7]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [15]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="how many users")

In [16]:
print(return_OAI(pqt1))

```json
{
    "tables": ["Users"]
}
```


In [17]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="product details")

In [18]:
print(return_OAI(pqt3))

```json
{
    "Tables": ["Products"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [19]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['Teachers', 'Courses', 'Students'], # ENTER A TABLE COLUMNS HERE,
        'definition':  ['Teacher name, The courses the teacher give',
                       'Courses names',
                       'Student name, the courses the student take, with which teacher']}# ENTER A TABLE DEFINITATIONS HERE

In [20]:
df = pd.DataFrame(data)
print(df)

text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])
print(text_tables)


      table                                         definition
0  Teachers         Teacher name, The courses the teacher give
1   Courses                                      Courses names
2  Students  Student name, the courses the student take, wi...
Teachers: Teacher name, The courses the teacher give
Courses: Courses names
Students: Student name, the courses the student take, with which teacher


In [21]:
df = pd.DataFrame(data)
print(df)

      table                                         definition
0  Teachers         Teacher name, The courses the teacher give
1   Courses                                      Courses names
2  Students  Student name, the courses the student take, wi...


In [22]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])
print(text_tables)

Teachers: Teacher name, The courses the teacher give
Courses: Courses names
Students: Student name, the courses the student take, with which teacher


In [23]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="the name of the teacher who gives math course")
print(return_OAI(pqt1))

```json
{
    "tables": ["Teachers", "Courses"]
}
```


In [24]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Return the Course name given by which teacher and the students taking the course")

print(return_OAI(pqt3))


{
    "tables": ["Teachers", "Courses", "Students"]
}
